# Intro to Data Science
## Part V. - Text Mining

### Table of contents
- #### Text Mining
    - <a href="#What-is-Text-Mining?">Theory</a>
    - <a href="#Text-Mining-in-practice">In practice</a>
    - <a href="#a)-Bag-of-words-representation">Vectorizing documents</a>
    - <a href="#b)-Tf-Idf">Normalizing document vectors</a>
    - <a href="#c)-Hashing">Vectorizing large corpus</a>
    - <a href="#3.-Latent-Semantic-Indexing">Topic modelling</a>
    - <a href="#4.-Document-similarity-metrics">Document similarity</a>
- #### ANN
    - <a href="#Neural-Networks">Single layer networks</a>
    - <a href="#Solving-non-linear-problems">Multi layer networks</a>
    

## What is Text Mining?
Text mining or text analytics is the process of extracting quantified features from (un)structured (natural language) texts. Processing unstructured data involves using Natural Language Processing (NLP), statistical modeling and machine learning techniques.

## Why is it important?
80% of the generated data is not available in structured, numerical format (emails, texts, meeting notes, documents, social media feeds). These unstructured data includes images, drawings, videos, voice recordings and unstructured texts. These data can be described with their meta data (length, topic, category, etc.) but transforming them into structured data is important to access the more detailed information stored in such data sources. Voice recordings, videos and drawings can also be transcribed into unstructured texts so they can be processing as textual data. 
Most common use cases are:

- document similarity computation
- document deduplication
- document clustering
- topic extraction
- sentiment analysis
- automated annotation
- text filtering
- text classification

## Tools
- NLP tools
    - tokenization
    - stemming
    - part-of-speech (POS) tagging
    - stop word filtering
    - bag-of-words representation
    - tf-idf transformation
- Other tools
    - Word2Vec representation
    - hashing
    - cosine/jaccard/levenshtein/etc similarity computation
    - matrix factorization
    - etc

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy.sparse as sp
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Text Mining in practice

### 1. Read and examine data

Examining unstructured data is the key to proper preprocessing.  
The collection of texts is called __`corpus`__.

In [ ]:
with open('./data/SMSSpamCollection') as spamfile:
    corpus = [line.strip() for line in spamfile]
len(corpus)

In [ ]:
for text in corpus[:5]:
    print text

We can see that the data is in TSV format, read it accordingly.

In [ ]:
corpus = pd.read_csv('./data/SMSSpamCollection', sep='\t', names=['label', 'message'])

In [ ]:
corpus.groupby('label').describe()

In [ ]:
corpus['length'] = corpus.message.str.len()
corpus.head()

In [ ]:
corpus.length.plot(bins=20, kind='hist')

In [ ]:
corpus.length.describe()

910 long sms???

In [ ]:
list(corpus.message[corpus.length > 900])

Is there a difference between spam and ham messages?

In [ ]:
corpus.hist(bins=50, by='label')

Why not try a simple predictor?

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
splitted = train_test_split(corpus.length.values[:, np.newaxis], # we need a matrix, not a vector
                            corpus.label.values,
                            train_size=.75,
                            random_state=42)
X_train, X_test, y_train, y_test = splitted

In [ ]:
pipe = Pipeline([('nb', MultinomialNB())])
pipe.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, pipe.predict(X_test))

87% percent is our baseline, let's get into the preprocessing!

### 2. Preprocessing
#### a) [Bag-of-words representation](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)

Bag of words representation represent documents as a vector where each different word is represented in a fixed position. The values in the positions are the occurence counts in the given document. For example:
The vector features for the documents:
```python
docs = ["I like trains.", "Trains are like big cars.", "I like big cars"]
```
will be 
```python
features = {'I': 0, 'like': 1, 'trains': 2, 'are': 3, 'big': 4, 'cars': 5}
```
and the vectorial representations will be
```python
vectors = [[1, 1, 1, 0, 0, 0],
           [0, 1, 1, 1, 1, 1],
           [1, 1, 0, 0, 1, 1]]
```

Fortunately `scikit-learn` has a built-in solution for this: the [`CountVectorizer`](http://scikit-learn.org/stable/modules/feature_extraction.html#the-bag-of-words-representation).  
Let's try out our little example:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cntvec = CountVectorizer()
docs = ["I like trains.",
        "Trains are like big cars.",
        "I like big cars"]
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

#### N-grams
N-grams are n long word tuples. They are generated by an n long rolling window and they can provide contextual information which sometimes yields better results. An example 2-gram for the `"I like trains."` sentence would be:
```python
[("I", "like"), ("like", "trains")]
```

In [ ]:
cntvec = CountVectorizer(ngram_range=(2, 2))
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

#### Minimum and maximum document frequency

Minimum and maximum document frequency (`min_df` and `max_df`) are set thresholds to limit the feature numbers. If a _term_ (transformed word) appears less than `min_df` or more than `max_df` times (or percent) then it will be left out from the vocabulary.

In [ ]:
cntvec = CountVectorizer(max_df=1)
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

In [ ]:
cntvec = CountVectorizer(min_df=3)
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

#### Lemmatization

In the vocabulary generation process each word are analyzed and transformed in order to reduce vocabulary length. Scikit-learn's default analization function is lowercasing the words and filtering short and stop words but no further transformation is done.

NLP has more detailed techniques to better extract the base words. Lemmatization is a powerful tool to reduce a word into it's _root_ form (as it appears in dictionaries): that's how `are` becomes `be` and `trains` becomes `train`, etc.

In [ ]:
import nltk
from textblob import TextBlob

In [ ]:
nltk.download('wordnet')

In [ ]:
def split_into_lemmas(message):
    message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    return [word.lemma for word in words]

[split_into_lemmas(doc) for doc in docs]

In [ ]:
cntvec = CountVectorizer(analyzer=split_into_lemmas)
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

Let's insert our vectorizer to our pipeline!

In [ ]:
splitted = train_test_split(corpus.message,
                            corpus.label.values,
                            train_size=.75,
                            random_state=42)
X_train, X_test, y_train, y_test = splitted

In [ ]:
pipe = Pipeline([('cntvec', CountVectorizer(analyzer=split_into_lemmas, min_df=10, max_df=.5)),
                 ('nb', MultinomialNB())])
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

In [ ]:
len(pipe.steps[0][-1].vocabulary_)

#### b) [Tf-Idf](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

Tf-Idf is short for Term Frequency - Inverse Document Frequency and is a way of normalizing term counts. It is a product of two separate metrics:

- _Term Frequency_ shows how often a word is appearing in a document. ${\displaystyle \mathrm {tf} (t,d)={\frac {1}{2}} + {\frac {f_{t,d}}{2 \cdot \max\{f_{t',d}:t'\in d\}}}}$
- _Inverse Document Frequency_ shows if a term is common or rare across all documents. $ \mathrm {idf}(t, D) =  \log \frac{N}{|\{d \in D: t \in d\}|}$ where $N$ is the total number of documents in the corpus, $t$ is a term, $D$ is the set of documents.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

In [ ]:
pipe = Pipeline([('cntvec', CountVectorizer(analyzer=split_into_lemmas, min_df=5, max_df=.9)),
                 ('tfidf', TfidfTransformer()),
                 ('nb', MultinomialNB())])
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

In [ ]:
for word in np.argsort(pipe.steps[1][1].idf_)[-20:][::-1]:
    print word, pipe.steps[0][1].get_feature_names()[word], pipe.steps[1][1].idf_[word]

#### c) [Hashing](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer)

Really large corpora induce several problems with required memory: the larger the corpus, the larger the vocabulary will grow in memory.
To avoid this issue a [_hashing trick_](http://scikit-learn.org/stable/modules/feature_extraction.html#feature-hashing) can be used. Basically instead of storing each different word in a dictionary it applies a hash function to the features to determine their column index in sample matrices directly.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
pipe = Pipeline([('hash', HashingVectorizer(analyzer=split_into_lemmas, n_features=1000, non_negative=True)),
                 ('nb', MultinomialNB())])
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

### 3. Latent Semantic Indexing

_"Latent semantic analysis (LSA) is a technique in natural language processing of analyzing relationships between a set of documents and the terms they contain by producing a set of concepts related to the documents and terms. LSA assumes that words that are close in meaning will occur in similar pieces of text."_ from: [wiki](https://en.wikipedia.org/wiki/Latent_semantic_analysis)  
LSA can be created by appling SVD to Tf-Idf vectors.

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC

In [ ]:
pipe = Pipeline([('cntvec', CountVectorizer(analyzer=split_into_lemmas, stop_words='english')),
                 ('tfidf', TfidfTransformer(sublinear_tf=True)),
                 ('svd', TruncatedSVD(n_components=300, random_state=42)),
                 ('svm', SVC(C=300))])
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

In [ ]:
feat_names = pipe.steps[0][1].get_feature_names()
topics = pipe.steps[2][1].components_
topic_str = pipe.steps[2][1].explained_variance_ratio_

In [ ]:
def get_most_important(topic, feat_names):
    indeces = np.argsort(topic)[::-1]
    terms = [feat_names[weightIndex] for weightIndex in indeces[:10]]    
    weights = [topic[weightIndex] for weightIndex in indeces[:10]] 
    return dict(zip(terms, weights))

In [ ]:
for i in range(10):
    print i, topic_str[i], get_most_important(topics[i], feat_names)
    print '-' * 80

### 4. Document similarity metrics

The euclidean distance is not feasable to determine the likeness of documents. Instead the cosine similarity is used which is the angle between the document vectors.

It is super convenient that the cosine similarity can be computed by the dot product between document tfidf vectors.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(analyzer=split_into_lemmas,
                        stop_words='english',
                        min_df=10,
                        max_df=.5).fit(corpus.message)
vects = tfidf.transform(corpus.message)

In [ ]:
vect = tfidf.transform([corpus.message[0]])

In [ ]:
sims = vects.dot(vect.T).toarray().flatten()
most_similar = np.argsort(sims)[::-1][:10]
for i, index in enumerate(most_similar):
    print i, sims[index]
    print corpus.message[index]
    print '-' * 80

## Model of the week:
### Neural Networks

<img src ="https://upload.wikimedia.org/wikibooks/en/thumb/1/17/Mcculloch_pitts.svg/400px-Mcculloch_pitts.svg.png" width="200" align="left"/>

Artificial Neural Networks are a supervised machine learning method for classification and regression purpose. It is based on the inner workings of the (human) brain. It consists of basic execution units and the connections between them.  

The execution units are called __neurons__, and their job is to compute the weighted summation of their inputs, then appling an output function. Based on their simple nature a [neuron](https://en.wikipedia.org/wiki/Perceptron) is only capable of solving linear problems. Their mechanism is easily expressed by the following equation:
$$y_{i} = f(\sum_{i}w(t)_{i}x_{i})$$

The learning process is simply updating the input weights: 
$${\displaystyle w_{i}(t+1)=w_{i}(t)+(d_{j}-y_{j}(t))x_{j,i}\,}$$
where $d$ is the expected output for the $j$th input.

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
XOR_X, XOR_y = np.array([[0,0], [0, 1], [1, 0], [1, 1]]), np.array([0, 1, 1, 0])
df = pd.DataFrame(data=XOR_X, columns=['x', 'y'])
df['label'] = XOR_y

In [ ]:
def plot_results_with_hyperplane(clf, clf_name, df, ax):
    x_min, x_max = df.x.min() - .5, df.x.max() + .5
    y_min, y_max = df.y.min() - .5, df.y.max() + .5

    xx, yy = np.meshgrid(np.arange(x_min, x_max, .02), np.arange(y_min, y_max, .02))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
    ax.scatter(df.x, df.y, c=df.label, edgecolors='k')
    ax.set_title(clf_name)

In [ ]:
perceptron = Perceptron(verbose=2, random_state=42).fit(XOR_X, XOR_y)
perceptron

In [ ]:
fig, ax = plt.subplots()
plot_results_with_hyperplane(perceptron, 'perceptron', df, ax)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(XOR_y, perceptron.predict(XOR_X))
conf_mat

In [ ]:
sns.heatmap(conf_mat)

#### Solving non-linear problems

As we can see, a single neuron is not able to solve this (not linear) problem. But they are not called __networks__ for nothing! The power of artificial neural networks lies in their topology. If we connect more of the __neurons__ we get a (real) neural network. The neurons are ordered into __layer__s. The first layer is the __input layer__ then there are zero, or more __hidden layer__(s), finally there is the __output layer__. Each layer can cosist any number of neurons. Based on the different topologies there are different ANN subtypes.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Artificial_neural_network.svg/560px-Artificial_neural_network.svg.png" width=200 align="left"/>

The most simple version of ANNs is the multi layer feed forward perceptron network (MLP). The tipical output (activation) functions are $y(v_i) = \tanh(v_i) ~~ \textrm{and} ~~ y(v_i) = (1+e^{-v_i})^{-1}$

The weight updating algorithm is called [__Backpropagation__](https://en.wikipedia.org/wiki/Backpropagation). It basically propagates the errors back to their "root" neuron. So every neuron which was (even slightly) responsible for an error will get their weights updated accordingly. The main updating equations are easily expressed by appling [gradient descent rules](https://en.wikipedia.org/wiki/Backpropagation#Derivation).

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier(random_state=42).fit(XOR_X, XOR_y)
mlp

In [ ]:
fig, ax = plt.subplots()
plot_results_with_hyperplane(mlp, 'mlp', df, ax)

In [ ]:
conf_mat = confusion_matrix(XOR_y, mlp.predict(XOR_X))
conf_mat

A super nice tutorial can be found [here](https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch12/ch12.ipynb), it is worth checking out.